In [39]:
import pandas as pd
import numpy as np
import torch
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.ensemble import IsolationForest
from tqdm import tqdm
import torch.optim as optim


# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [40]:
import sys

sys.path.append('../anomalib')
from anomalib.data import MVTec
from anomalib.models import Patchcore
from anomalib.engine import Engine
from anomalib.data import Folder
from anomalib.data.utils import TestSplitMode


In [3]:
# Create the datamodule
datamodule = Folder(
    name="abc",
    root="",
    normal_dir="train",
    test_split_mode=TestSplitMode.SYNTHETIC,
)

# Setup the datamodule
datamodule.setup()

In [41]:
# Create the datamodule
datamodule_test = Folder(
    name="abc",
    root="",
    normal_dir="train",
    # normal_test_dir=  "test",
    task = 'classification',
)

# Setup the datamodule
datamodule.setup()

In [38]:
datamodule_test

In [42]:
# Initialize the datamodule, model and engine
model = Patchcore()
engine = Engine(task = "classification",default_root_dir='results')

In [43]:
datamodule

In [44]:
# Train the model
engine.fit(datamodule=datamodule_test, model=model)

C:\Users\SJ\anaconda3\envs\anomalib\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\SJ\anaconda3\envs\anomalib\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `ROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\SJ\anaconda3\envs\anomalib\lib\site-packages\lightning\pytorch\core\optimizer.py:180: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                  | Type                     | Params
-------------------------------------------------------------------


Training: |          | 0/? [00:00<?, ?it/s]

C:\Users\SJ\anaconda3\envs\anomalib\lib\site-packages\lightning\pytorch\loops\optimization\automatic.py:129: `training_step` returned `None`. If this was on purpose, ignore this warning...


Validation: |          | 0/? [00:00<?, ?it/s]

Output()

C:\Users\SJ\anaconda3\envs\anomalib\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [56]:
from anomalib.data.utils import read_image
from matplotlib import pyplot as plt
from pathlib import Path
from anomalib.deploy import ExportType, OpenVINOInferencer

In [66]:
engine.export(
    model=model,
    export_type=ExportType.OPENVINO,
)

WindowsPath('C:/Users/SJ/Documents/code/test/code/results/Patchcore/abc/latest/weights/openvino/model.xml')

In [98]:
image_path ="test/TEST_006.png"

In [67]:
output_path = Path(engine.trainer.default_root_dir)
print(output_path)


C:\Users\SJ\Documents\code\test\code\results\Patchcore\abc\latest


In [68]:
openvino_model_path = output_path / "weights" / "openvino" / "model.bin"
metadata = output_path / "weights" / "openvino" / "metadata.json"
print(openvino_model_path.exists(), metadata.exists())

True True


In [69]:
inferencer = OpenVINOInferencer(
    path=openvino_model_path,  # Path to the OpenVINO IR model.
    metadata=metadata,  # Path to the metadata file.
    device="CPU",  # We would like to run it on an Intel CPU.
)

In [100]:
predictions = inferencer.predict(image=image_path)

In [101]:
print(predictions.pred_score, predictions.pred_label)

0.8005872450743634 True


In [111]:
# 예측 레이블을 저장할 리스트 초기화
pred_labels = []
pred_score = []
# TEST_000.png부터 TEST_099.png까지 반복
for i in range(100):
    # 이미지 파일 이름 생성
    image_path = f"test/TEST_{i:03d}.png"

    # 이미지에 대해 추론 수행
    predictions = inferencer.predict(image=image_path)

    # pred_score가 0.6보다 크면 True, 그렇지 않으면 False를 리스트에 추가
    if predictions.pred_score > 0.6:
        pred_labels.append(True)
        pred_score.append(predictions.pred_score)
    else:
        pred_labels.append(False)
        pred_score.append(predictions.pred_score)

# 결과 출력
print(pred_labels)
converted_labels = [1 if label else 0 for label in pred_labels]

[False, True, False, False, False, False, True, False, False, True, False, True, False, False, True, True, False, False, False, False, False, False, True, True, True, False, True, False, False, True, True, True, False, False, True, False, False, False, True, True, False, False, False, False, True, False, False, True, False, False, False, False, False, True, False, True, False, True, False, False, False, False, False, True, False, False, True, False, False, True, False, False, False, False, True, False, False, True, False, True, False, True, False, False, False, True, True, False, True, False, True, True, True, True, True, True, False, False, True, True]


In [131]:
# 값이 0.4와 0.6 사이인 원소들의 인덱스 찾기
indices = [index for index, value in enumerate(pred_score) if 0.5 <= value <= 0.7]

# 찾은 인덱스 출력
print(indices)

[0, 25, 34, 38, 54, 83, 84, 91]


In [135]:
pred_score[84]

0.5018101939550735

In [105]:
submission = pd.read_csv("../submission/sample_submission.csv")
submission['label'] = converted_labels

In [106]:
submission

,id,label
0,TEST_000,0
1,TEST_001,1
2,TEST_002,0
3,TEST_003,0
4,TEST_004,0
...,...,...
95,TEST_095,1
96,TEST_096,0
97,TEST_097,0
98,TEST_098,1


In [107]:
submission.to_csv("../submission/submission_6.csv",index = False)